In [3]:
import os

os.listdir('data/gear_images')

['harnesses',
 'crampons',
 'rope',
 'insulated_jackets',
 'tents',
 'gloves',
 'hardshell_jackets',
 'carabiners',
 'boots',
 'axes',
 'pulleys',
 'helmets']

In [10]:
from azure.cognitiveservices.vision.customvision.training import training_api
from azure.cognitiveservices.vision.customvision.training.models import ImageUrlCreateEntry

In [21]:
# Replace with a valid key
training_key = "1d969e55dfd94cb3943ceeba93ee8f1c"
prediction_key = "98b69e204d444fa9a9e7497769eb7311"

trainer = training_api.TrainingApi(training_key)

project = trainer.get_project("5ee47e32-0f32-40ba-b4c4-2b18167a5f94")

In [22]:
hardshell_tag = trainer.create_tag(project.id, "hardshell")
insulated_tag = trainer.create_tag(project.id, "insulated")

In [26]:
import os
hardshell_dir = 'data/gear_images/hardshell_jackets'
for image in os.listdir(os.fsencode('data/gear_images/hardshell_jackets')):
    with open(hardshell_dir + "/" + os.fsdecode(image), mode="rb") as img_data: 
        trainer.create_images_from_data(project.id, img_data, [ hardshell_tag.id ])

insulated_dir = 'data/gear_images/insulated_jackets'
for image in os.listdir(os.fsencode('data/gear_images/insulated_jackets')):
    with open(insulated_dir + "/" + os.fsdecode(image), mode="rb") as img_data: 
        trainer.create_images_from_data(project.id, img_data, [ insulated_tag.id ])

In [45]:
import time

print ("Training...")
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(1)

# The iteration is now trained. Make it the default project endpoint
trainer.update_iteration(project.id, iteration.id, is_default=True)
print ("Done!")

Training...


HttpOperationError: Operation returned an invalid status code 'Bad Request'

In [63]:
hardshell_test_url = 'https://cdn.shopify.com/s/files/1/0622/2637/products/60008-Hard_Shell_Jacket-SO1_2CY_1024x1024.jpg?v=1430840037'
insulated_test_url = 'https://cdna.4imprint.com/prod/700/505374.jpg'

In [43]:
from azure.cognitiveservices.vision.customvision.prediction import prediction_endpoint
from azure.cognitiveservices.vision.customvision.prediction.prediction_endpoint import models

In [64]:
predictor = prediction_endpoint.PredictionEndpoint(prediction_key)

results = predictor.predict_image_url(project.id, iteration.id, url=hardshell_test_url)

print('Hardshell')
for prediction in results.predictions:
    print ("\t" + prediction.tag_name + ": {0:.2f}%".format(prediction.probability * 100))
    
predictor = prediction_endpoint.PredictionEndpoint(prediction_key)

results = predictor.predict_image_url(project.id, iteration.id, url=insulated_test_url)

print('Insulated')
for prediction in results.predictions:
    print ("\t" + prediction.tag_name + ": {0:.2f}%".format(prediction.probability * 100))

Hardshell
	insulated: 55.55%
	hardshell: 44.45%
Insulated
	insulated: 100.00%
	hardshell: 0.00%
